# Counting with Crosstabs

In this chapter, we explore the pandas `crosstab` function, which produces very similar results as the `pivot_table` method, but allows us to count occurrences in a cleaner way and with more functionality.

### Exploring mental health survey data

We will be using mental health survey data found on [Kaggle datasets][1]. This dataset is from a 2014-2015 survey that measured the attitude towards mental health and frequency of mental health disorders in the tech workplace. Let's read in the dataset and output the number of rows and columns.

[1]: https://www.kaggle.com/osmi/mental-health-in-tech-survey

In [11]:
import pandas as pd
mh = pd.read_csv('../data/mental_health.csv')
mh.head(3)

,year,age,gender,country,family_history,treatment,work_interfere,no_employees,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor
0,2014,37,Female,United States,No,Yes,Often,6-25,Yes,Yes,Not sure,No,Yes,Yes,Somewhat easy,No,No,Some of them,Yes
1,2014,44,Male,United States,No,No,Rarely,More than 1000,No,Don't know,No,Don't know,Don't know,Don't know,Don't know,Maybe,No,No,No
2,2014,32,Male,Canada,No,No,Rarely,6-25,Yes,No,No,No,No,Don't know,Somewhat difficult,No,No,Yes,Yes


In [10]:
mh.shape

(1091, 19)

### Data Dictionary

The data dictionary will help you understand the questions asked behind the data collected on each column. Some of the column descriptions are larger than the default 50 character setting. We change the option `'display.max_colwidth'` before outputting the data dictionary.

In [12]:
pd.set_option('display.max_colwidth', 100)
mh_dd = pd.read_csv('../data/dictionaries/mental_health_dd.csv')
mh_dd

,Column Name,Description
0,year,Year the survey was submitted
1,age,Respondent age
2,gender,Respondent gender
3,country,Respondent country
4,family_history,Do you have a family history of mental illness?
5,treatment,Have you sought treatment for a mental health condition?
6,work_interfere,Does your mental health condition interfere with your work (if you have one)?
7,no_employees,How many employees does your company or organization have?
8,tech_company,Is your employer primarily a tech company/organization?
9,benefits,Does your employer provide mental health benefits?


### Converting columns to categorical

All of the questions appear to have a limited number of discrete answer choices. Here, we find the number of unique values in each column is limited, with age being the only exception.

In [13]:
mh.nunique()

year                          2
age                          49
gender                        2
country                       8
family_history                2
treatment                     2
work_interfere                4
no_employees                  6
tech_company                  2
benefits                      3
care_options                  3
wellness_program              3
seek_help                     3
anonymity                     3
leave                         5
mental_health_consequence     3
phys_health_consequence       3
coworkers                     3
supervisor                    3
dtype: int64

Let's convert all of the string columns (only year and age are numeric) to categorical. Instead of converting each column individually, we can convert an entire selection of the DataFrame and overwrite the old columns at the same time. This will save memory and help performance when grouping.

In [15]:
for col in mh.loc[:, 'gender':].columns:
    mh[col] = mh[col].astype('category')

In [16]:
#mh.loc[:, 'gender':] = mh.loc[:, 'gender':].astype('category')
mh.dtypes

year                            int64
age                             int64
gender                       category
country                      category
family_history               category
treatment                    category
work_interfere               category
no_employees                 category
tech_company                 category
benefits                     category
care_options                 category
wellness_program             category
seek_help                    category
anonymity                    category
leave                        category
mental_health_consequence    category
phys_health_consequence      category
coworkers                    category
supervisor                   category
dtype: object

## Frequency counting with a Series

Previously, we learned how to count the frequency of values of a single column of data as a Series with the `value_counts` method. Let's review this by finding the number of survey respondents by country. 

In [17]:
mh['country'].value_counts()

country
United States     717
United Kingdom    177
Canada             68
Germany            43
Netherlands        27
Ireland            27
Australia          21
France             11
Name: count, dtype: int64

The relative frequencies are returned by setting `normalize` to `True`.

In [18]:
mh['country'].value_counts(normalize=True).round(3)

country
United States     0.657
United Kingdom    0.162
Canada            0.062
Germany           0.039
Netherlands       0.025
Ireland           0.025
Australia         0.019
France            0.010
Name: proportion, dtype: float64

## Counting the mental health occurrences by country

If we are interested in counting the co-occurrence of values appearing in two or more columns, we can use the DataFrame `value_counts`, `groupby`, or `pivot_table` methods. Let's see examples of counting the occurrences of seeking mental health treatment (the `'treatment'` column) by country.

### Counting frequency with  `groupby`

Use both country and treatment as grouping columns and then aggregate any column with `size`.

In [21]:
mh.head(3)

,year,age,gender,country,family_history,treatment,work_interfere,no_employees,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor
0,2014,37,Female,United States,No,Yes,Often,6-25,Yes,Yes,Not sure,No,Yes,Yes,Somewhat easy,No,No,Some of them,Yes
1,2014,44,Male,United States,No,No,Rarely,More than 1000,No,Don't know,No,Don't know,Don't know,Don't know,Don't know,Maybe,No,No,No
2,2014,32,Male,Canada,No,No,Rarely,6-25,Yes,No,No,No,No,Don't know,Somewhat difficult,No,No,Yes,Yes


In [20]:
mh.groupby(['country', 'treatment'], observed=False).agg(count=('age', 'size'))

count
country        treatment       
Australia      No             8
               Yes           13
Canada         No            33
               Yes           35
France         No            11
               Yes            0
Germany        No            24
               Yes           19
Ireland        No            14
               Yes           13
Netherlands    No            18
               Yes            9
United Kingdom No            90
               Yes           87
United States  No           327
               Yes          390

### Unobserved categories still appear in the result.

Even if one of the combinations of country and treatment does not exist (such as France and Yes), it will still appear in the result. This is because both of these columns were converted to categorical and all categories, by default, appear in the result. To change this behavior so that only those groups that are **observed**, set the `observed` parameter to `True`. The result will now not include France with Yes, reducing the total number of groups from 16 to 15.

In [22]:
len(mh.groupby(['country', 'treatment'], observed=True).size())

15

### Counting frequency with `pivot_table`

Alternatively, we can count frequencies using `pivot_table` just how we did in the previous chapter.

In [23]:
mh.pivot_table(index='country', columns='treatment', aggfunc='size')

C:\Users\nealv\AppData\Local\Temp\ipykernel_15036\517226056.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  mh.pivot_table(index='country', columns='treatment', aggfunc='size')


treatment,No,Yes
country,,
Australia,8,13
Canada,33,35
France,11,0
Germany,24,19
Ireland,14,13
Netherlands,18,9
United Kingdom,90,87
United States,327,390


## Counting frequency with the `crosstab` function

The `crosstab` function is built specifically for the situation of counting co-occurrences of values between two or more columns. The name comes from **cross tabulation** which is the more generic term used in data analysis outside of pandas. They are also known as [contingency tables][1].

Unfortunately, `crosstab` is a function and NOT a method. This means it is not bound to any DataFrame object, but must be accessed directly from `pd`. It has many of the same parameter names as the `pivot_table` method and is used similarly. Since it is not bound to any DataFrame object, you must set its parameters to Series and not strings. By default, it will compute the size of each group so there is no need to set the `aggfunc` parameter.

[1]: https://en.wikipedia.org/wiki/Contingency_table

In [ ]:
pd.crosstab(index=mh['country'], columns=mh['treatment'])

treatment,No,Yes
country,,
Australia,8,13
Canada,33,35
France,11,0
Germany,24,19
Ireland,14,13
Netherlands,18,9
United Kingdom,90,87
United States,327,390


### Relative frequencies - only available with `crosstab`

The result is identical to what the `pivot_table` method produced. You might be wondering why there is a need to even know about this function. There is one big benefit of using the `crosstab` function, and that is its ability to return relative frequencies with the `normalize` parameter. This isn't easily doable with `groupby` or `pivot_table`. The `crosstab` function allows you to normalize over the rows, columns, or all of the data. For instance, to find the relative frequency of people who have sought treatment in each country, you can normalize across each row like this. The rows should all sum to 100%.

In [25]:
pd.crosstab(index=mh['country'], 
            columns=mh['treatment'], 
            normalize='index').round(3) * 100

treatment,No,Yes
country,,
Australia,38.1,61.9
Canada,48.5,51.5
France,100.0,0.0
Germany,55.8,44.2
Ireland,51.9,48.1
Netherlands,66.7,33.3
United Kingdom,50.8,49.2
United States,45.6,54.4


Set the `normalize` parameter to the string `'columns'` to return the relative frequency in the other direction. The returned DataFrame informs us that of all the respondents seeking treatment, 15.4% were from the United Kingdom.

In [26]:
pd.crosstab(index=mh['country'], 
            columns=mh['treatment'], 
            normalize='columns').round(3) * 100

treatment,No,Yes
country,,
Australia,1.5,2.3
Canada,6.3,6.2
France,2.1,0.0
Germany,4.6,3.4
Ireland,2.7,2.3
Netherlands,3.4,1.6
United Kingdom,17.1,15.4
United States,62.3,68.9


It's possible to find the relative frequency against all of the data by setting the `normalize` parameter to `'all'`. From the returned DataFrame, 2.2% of all respondents are Germans who have not received mental health treatment.

In [27]:
pd.crosstab(index=mh['country'], 
            columns=mh['treatment'], 
            normalize='all').round(3) * 100

treatment,No,Yes
country,,
Australia,0.7,1.2
Canada,3.0,3.2
France,1.0,0.0
Germany,2.2,1.7
Ireland,1.3,1.2
Netherlands,1.6,0.8
United Kingdom,8.2,8.0
United States,30.0,35.7


### Adding margins

You can add margins as well by setting the `margins` parameter to `True`. Here, we go back to raw counts and add margins for all rows and columns.

In [29]:
pd.crosstab(index=mh['country'], columns=mh['treatment'], margins=True).round(3)

treatment,No,Yes,All
country,,,
Australia,8,13,21
Canada,33,35,68
France,11,0,11
Germany,24,19,43
Ireland,14,13,27
Netherlands,18,9,27
United Kingdom,90,87,177
United States,327,390,717
All,525,566,1091


When normalizing the data, the margins calculated depend on the direction of the normalization. Here, we add margins when normalizing down the columns. We can use this margin to determine the degree to which each country is overrepresented (or underrepresented) in each treatment category. For instance, 65.7% of the respondents were from the United States. Of those respondents seeking treatment, 68.9% were from the United States informing us that respondents from the United States were overrepresented in that category.

In [30]:
pd.crosstab(index=mh['country'], columns=mh['treatment'], 
            normalize='columns', margins=True).round(3)

treatment,No,Yes,All
country,,,
Australia,0.015,0.023,0.019
Canada,0.063,0.062,0.062
France,0.021,0.000,0.010
Germany,0.046,0.034,0.039
Ireland,0.027,0.023,0.025
Netherlands,0.034,0.016,0.025
United Kingdom,0.171,0.154,0.162
United States,0.623,0.689,0.657


## Normalizing other aggregations

While `pd.crosstab` is most often used for frequency, it's possible to supply it another column to aggregate. Let's read in the City of Houston dataset for this example.

In [31]:
emp = pd.read_csv('../data/employee.csv')
emp.head(3)

,dept,title,hire_date,salary,sex,race
0,Police,POLICE SERGEANT,2001-12-03,87545.38,Male,White
1,Other,ASSISTANT CITY ATTORNEY II,2010-11-15,82182.00,Male,Hispanic
2,Houston Public Works,SENIOR SLUDGE PROCESSOR,2006-01-09,49275.00,Male,Black


Here, we calculate the total salary by department and sex with `pd.crosstab` passing the aggregating column, `salary`, as a Series to the `values` parameter and setting `aggfunc` to the string aggregation name, `'sum`'.

In [32]:
pd.crosstab(index=emp['dept'], columns=emp['sex'],
            values=emp['salary'], aggfunc='sum')

sex,Female,Male
dept,,
Fire,1.493103e+07,2.501424e+08
Health & Human Services,5.313841e+07,2.167834e+07
Houston Airport System,2.263699e+07,4.427613e+07
Houston Public Works,6.129687e+07,1.542129e+08
Library,1.661529e+07,7.059591e+06
Other,1.113827e+08,9.586310e+07
Parks & Recreation,1.345082e+07,2.921983e+07
Police,9.853668e+07,3.429797e+08
Solid Waste Management,6.022059e+06,1.638348e+07


This is typically done with the `pivot_table` method, which produces the exact same result.

In [33]:
emp.pivot_table(index='dept', columns='sex', values='salary', aggfunc='sum')

sex,Female,Male
dept,,
Fire,1.493103e+07,2.501424e+08
Health & Human Services,5.313841e+07,2.167834e+07
Houston Airport System,2.263699e+07,4.427613e+07
Houston Public Works,6.129687e+07,1.542129e+08
Library,1.661529e+07,7.059591e+06
Other,1.113827e+08,9.586310e+07
Parks & Recreation,1.345082e+07,2.921983e+07
Police,9.853668e+07,3.429797e+08
Solid Waste Management,6.022059e+06,1.638348e+07


As we saw above, `crosstab` has the ability to normalize over rows, columns, and the entire table. It's possible to normalize over any aggregation provided, not just frequency (the default). Here, we find the percentage of the total female (and male) salaries by department by setting `normalize` to `'columns'`. It informs us, that out of the total of female salaries, 3.8% are from the fire department, 24.8% are from the police department, etc... It provides a distribution of each column over the index. The columns total to 100%.

In [34]:
pd.crosstab(index=emp['dept'], columns=emp['sex'],
            values=emp['salary'], aggfunc='sum', 
            normalize='columns').round(3) * 100

sex,Female,Male
dept,,
Fire,3.8,26.0
Health & Human Services,13.4,2.3
Houston Airport System,5.7,4.6
Houston Public Works,15.4,16.0
Library,4.2,0.7
Other,28.0,10.0
Parks & Recreation,3.4,3.0
Police,24.8,35.7
Solid Waste Management,1.5,1.7


### `crosstab` is almost unnecessary in pandas

It's important to know that `crosstab` and `pivot_table` are very similar and `crosstab` would be unnecessary if `pivot_table` had an easy way to normalize the values across groups. Since it does not, `crosstab` is still valuable.

## Exercises

### Exercise 1
<span  style="color:green; font-size:16px">Do people with a family history of mental illness seek treatment more often than those who do not?</span>

In [36]:
mh.head()

,year,age,gender,country,family_history,treatment,work_interfere,no_employees,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor
0,2014,37,Female,United States,No,Yes,Often,6-25,Yes,Yes,Not sure,No,Yes,Yes,Somewhat easy,No,No,Some of them,Yes
1,2014,44,Male,United States,No,No,Rarely,More than 1000,No,Don't know,No,Don't know,Don't know,Don't know,Don't know,Maybe,No,No,No
2,2014,32,Male,Canada,No,No,Rarely,6-25,Yes,No,No,No,No,Don't know,Somewhat difficult,No,No,Yes,Yes
3,2014,31,Male,United Kingdom,Yes,Yes,Often,26-100,Yes,No,Yes,No,No,No,Somewhat difficult,Yes,Yes,Some of them,No
4,2014,31,Male,United States,No,No,Never,100-500,Yes,Yes,No,Don't know,Don't know,Don't know,Don't know,No,No,Some of them,Yes


In [49]:
mh_fh = mh.query('family_history == "Yes" ')

pd.crosstab(index=mh_fh['family_history'], columns=mh_fh['treatment'])

treatment,No,Yes
family_history,,
Yes,111,325


### Exercise 2
<span  style="color:green; font-size:16px">Find the total number and ratio of employees that seek treatment for companies that provide health benefits vs those that do not.</span>

In [46]:
mh.head(3)

,year,age,gender,country,family_history,treatment,work_interfere,no_employees,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor
0,2014,37,Female,United States,No,Yes,Often,6-25,Yes,Yes,Not sure,No,Yes,Yes,Somewhat easy,No,No,Some of them,Yes
1,2014,44,Male,United States,No,No,Rarely,More than 1000,No,Don't know,No,Don't know,Don't know,Don't know,Don't know,Maybe,No,No,No
2,2014,32,Male,Canada,No,No,Rarely,6-25,Yes,No,No,No,No,Don't know,Somewhat difficult,No,No,Yes,Yes


In [50]:
pd.crosstab(index=mh['benefits'], columns=mh['treatment'], normalize=True)

treatment,No,Yes
benefits,,
Don't know,0.206233,0.122823
No,0.130156,0.137489
Yes,0.144821,0.258478


### Exercise 3
<span  style="color:green; font-size:16px">You can provide a list of multiple columns to both the `index` and `columns` parameters of the `crosstab` function. Put country and number of employees in the index and benefits and treatment in the columns. It's probably easier to make separate list variables first.</span>

In [51]:
mh.head(3)

,year,age,gender,country,family_history,treatment,work_interfere,no_employees,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor
0,2014,37,Female,United States,No,Yes,Often,6-25,Yes,Yes,Not sure,No,Yes,Yes,Somewhat easy,No,No,Some of them,Yes
1,2014,44,Male,United States,No,No,Rarely,More than 1000,No,Don't know,No,Don't know,Don't know,Don't know,Don't know,Maybe,No,No,No
2,2014,32,Male,Canada,No,No,Rarely,6-25,Yes,No,No,No,No,Don't know,Somewhat difficult,No,No,Yes,Yes


In [52]:
pd.crosstab(index=[mh['country'],mh['no_employees']], columns=[mh['benefits'],mh['treatment']])

benefits                      Don't know      No     Yes    
treatment                             No Yes  No Yes  No Yes
country        no_employees                                 
Australia      1-5                     1   0   1   1   0   0
               100-500                 1   0   1   2   0   2
               26-100                  0   0   1   3   0   0
               500-1000                1   0   0   0   0   0
               6-25                    0   1   0   3   0   0
               More than 1000          1   0   0   0   1   1
Canada         1-5                     1   0   5   5   0   0
               100-500                 2   3   0   0   1   3
               26-100                  4   4   2   1   3   3
               500-1000                0   0   0   0   0   1
               6-25                    4   2   6   1   3   5
               More than 1000          0   2   0   0   2   5
France         100-500                 1   0   1   0   0   0
               26-100                  0   0   3   0   0   0
               500-1000                1   0   0   0   1   0
               6-25                    1   0   3   0   0   0
Germany        1-5                     0   0   3   3   0   1
               100-500                 1   2   1   0   0   0
               26-100                  2   4   4   3   1   0
               500-1000                1   0   0   0   1   0
               6-25                    5   1   3   2   0   2
               More than 1000          2   1   0   0   0   0
Ireland        1-5                     1   0   5   4   0   0
               100-500                 1   0   0   1   0   0
               26-100                  0   1   2   3   0   0
               500-1000                0   0   0   0   0   1
               6-25                    0   0   1   2   0   0
               More than 1000          2   0   1   0   1   1
Netherlands    1-5                     2   2   1   2   0   1
               100-500                 1   0   0   0   1   1
               26-100                  1   0   2   1   0   0
               500-1000                1   0   0   0   0   0
               6-25                    1   1   7   1   0   0
               More than 1000          0   0   0   0   1   0
United Kingdom 1-5                     6   3   7  13   0   1
               100-500                 5   6   2   3   2   1
               26-100                 11   5   7  10   3   2
               500-1000                2   3   1   0   1   0
               6-25                   11   7  23  13   1   1
               More than 1000          5   4   3   7   0   8
United States  1-5                    11   4  16  30   3   8
               100-500                21  14   3   8  23  41
               26-100                 41  26   7   6  18  61
               500-1000                8   1   1   1   9  20
               6-25                   30  22  14  19  17  25
               More than 1000         35  15   5   2  65  87

In [64]:
mh['no_employees'].unique()

['6-25', 'More than 1000', '26-100', '100-500', '1-5', '500-1000']
Categories (6, object): ['1-5', '100-500', '26-100', '500-1000', '6-25', 'More than 1000']

In [65]:
no_emp_order = ['1-5','6-25','26-100', '100-500', '500-1000','More than 1000']

no_emp_cat = pd.CategoricalDtype(no_emp_order, ordered=True)

mh['no_employees'] = mh['no_employees'].astype(no_emp_cat)

In [66]:
index_val = [mh['country'],mh['no_employees']]

col_val = [mh['benefits'],mh['treatment']]

pd.crosstab(index=index_val, columns=col_val)

benefits                      Don't know      No     Yes    
treatment                             No Yes  No Yes  No Yes
country        no_employees                                 
Australia      1-5                     1   0   1   1   0   0
               6-25                    0   1   0   3   0   0
               26-100                  0   0   1   3   0   0
               100-500                 1   0   1   2   0   2
               500-1000                1   0   0   0   0   0
               More than 1000          1   0   0   0   1   1
Canada         1-5                     1   0   5   5   0   0
               6-25                    4   2   6   1   3   5
               26-100                  4   4   2   1   3   3
               100-500                 2   3   0   0   1   3
               500-1000                0   0   0   0   0   1
               More than 1000          0   2   0   0   2   5
France         6-25                    1   0   3   0   0   0
               26-100                  0   0   3   0   0   0
               100-500                 1   0   1   0   0   0
               500-1000                1   0   0   0   1   0
Germany        1-5                     0   0   3   3   0   1
               6-25                    5   1   3   2   0   2
               26-100                  2   4   4   3   1   0
               100-500                 1   2   1   0   0   0
               500-1000                1   0   0   0   1   0
               More than 1000          2   1   0   0   0   0
Ireland        1-5                     1   0   5   4   0   0
               6-25                    0   0   1   2   0   0
               26-100                  0   1   2   3   0   0
               100-500                 1   0   0   1   0   0
               500-1000                0   0   0   0   0   1
               More than 1000          2   0   1   0   1   1
Netherlands    1-5                     2   2   1   2   0   1
               6-25                    1   1   7   1   0   0
               26-100                  1   0   2   1   0   0
               100-500                 1   0   0   0   1   1
               500-1000                1   0   0   0   0   0
               More than 1000          0   0   0   0   1   0
United Kingdom 1-5                     6   3   7  13   0   1
               6-25                   11   7  23  13   1   1
               26-100                 11   5   7  10   3   2
               100-500                 5   6   2   3   2   1
               500-1000                2   3   1   0   1   0
               More than 1000          5   4   3   7   0   8
United States  1-5                    11   4  16  30   3   8
               6-25                   30  22  14  19  17  25
               26-100                 41  26   7   6  18  61
               100-500                21  14   3   8  23  41
               500-1000                8   1   1   1   9  20
               More than 1000         35  15   5   2  65  87

### Exercise 4

<span style="color:green; font-size:16px">Read in the bikes dataset and find the distribution of total trip duration by gender and events. Normalize over all groups. You should be able to answer the question, "From the total of all trip durations, what percent were done by males on a clear day?".</span>

In [55]:
bikes = pd.read_csv('../data/bikes.csv')

In [56]:
bikes.head(3)

,gender,starttime,stoptime,tripduration,from_station_name,start_capacity,to_station_name,end_capacity,temperature,wind_speed,events
0,Male,2013-06-28 19:01:00,2013-06-28 19:17:00,993,Lake Shore Dr & Monroe St,11.0,Michigan Ave & Oak St,15.0,73.9,12.7,mostlycloudy
1,Male,2013-06-28 22:53:00,2013-06-28 23:03:00,623,Clinton St & Washington Blvd,31.0,Wells St & Walton St,19.0,69.1,6.9,partlycloudy
2,Male,2013-06-30 14:43:00,2013-06-30 15:01:00,1040,Sheffield Ave & Kingsbury St,15.0,Dearborn St & Monroe St,23.0,73.0,16.1,mostlycloudy


In [68]:
pd.crosstab(index=bikes['gender'], columns=bikes['events'], values=bikes['tripduration'], aggfunc='sum', normalize='columns', margins=True).round(3)*100

events,clear,cloudy,fog,hazy,mostlycloudy,partlycloudy,rain,sleet,snow,tstorms,unknown,All
gender,,,,,,,,,,,,
Female,27.2,26.2,30.1,28.2,28.0,30.3,22.6,27.1,14.2,28.1,9.4,28.0
Male,72.8,73.8,69.9,71.8,72.0,69.7,77.4,72.9,85.8,71.9,90.6,72.0
